In [ ]:
# Importing necessary libraries
import cv2
import os
import csv
import mediapipe as mp

In [ ]:
# Paths and parameters
IMAGES_PATH = "images/collected"
KEYPOINTS_PATH = "images/keypoints"
labels = ["hello", "thanks", "yes", "no", "iloveyou"]

In [ ]:
# Initialize MediaPipe hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

In [ ]:
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def save_keypoints_to_csv(label):
    """
    Loads images for the given label, detects hand landmarks, and saves keypoints to a CSV file.
    """
    label_img_path = os.path.join(IMAGES_PATH, label)
    keypoints_file = os.path.join(KEYPOINTS_PATH, f"{label}_keypoints.csv")
    
    # Prepare CSV file with headers for each keypoint (x, y, z coordinates for 21 landmarks)
    with open(keypoints_file, mode='w', newline='') as f:
        writer = csv.writer(f)
        headers = ["image_name"]
        headers += [f"{axis}_{i}" for i in range(21) for axis in ("x", "y", "z")]
        writer.writerow(headers)

        for image_file in os.listdir(label_img_path):
            image_path = os.path.join(label_img_path, image_file)

            # Read image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Could not load image {image_path}")
                continue

            # Convert the BGR image to RGB for MediaPipe
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb_image)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Flatten the landmark coordinates into a single list
                    keypoints_row = [image_file]
                    keypoints_row += [coord for landmark in hand_landmarks.landmark for coord in (landmark.x, landmark.y, landmark.z)]
                    writer.writerow(keypoints_row)
                    print(f"Saved keypoints for {image_path} to CSV")
            else:
                print(f"No hand detected in image {image_path}")

In [ ]:
def main():
    """
    Extract keypoints for each label's images and save to CSV files.
    """
    create_directory(KEYPOINTS_PATH)

    for label in labels:
        save_keypoints_to_csv(label)

if __name__ == "__main__":
    main()